In [1]:
# necessary library imports
import requests
from pathlib import Path
import pandas as pd
import json
import datetime
import sys
import zlib
import msgpack
import tqdm
import time
import os.path
# import helper_functions as hf
# import io_helper as ioh
# import files from src
sys.path.insert(0, "src")

In [2]:
# define where we're going to save the data
path_save_data = "Project_2"

# Check whether the specified path exists or not
isExist = os.path.exists(path_save_data)
if not isExist:
    # Create a new directory because it does not exist 
    os.makedirs(path_save_data)
    print("The new directory is created!")

filename_save_data = "{:s}/augmento_data.msgpack.zlib".format(path_save_data)


print(isExist)

True


In [3]:
# define the url of the endpoint to get event data
endpoint_url = "http://api-dev.augmento.ai/v0.1/events/aggregated"


# define the start and end times
datetime_start = datetime.datetime(2017, 1, 1)
datetime_end = datetime.datetime(2022, 6, 1)

# initialise a store for the data we're downloading
sentiment_data = []

# define a start pointer to track multiple requests
start_ptr = 0
count_ptr = 1000

# get the data
while start_ptr >= 0:
	
	# define the parameters of the request
	params = {
		"source" : "twitter",
		"coin" : "ethereum",
		"bin_size" : "24H",
		"count_ptr" : count_ptr,
		"start_ptr" : start_ptr,
		"start_datetime" : datetime_start.strftime("%Y-%m-%dT%H:%M:%SZ"),
		"end_datetime" : datetime_end.strftime("%Y-%m-%dT%H:%M:%SZ"),
	}
	
	# make the request
	r = requests.request("GET", endpoint_url, params=params, timeout=10)
	
	# if the request was ok, add the data and increment the start_ptr
	# else return an error
	if r.status_code == 200:
		temp_data = r.json()
		start_ptr += count_ptr
	else:
		raise Exception("api call failed with status_code {:d}".format(r.status_code))
	
	# if we didn't get any data, assume we've got all the data
	if len(temp_data) == 0:
		start_ptr = -1
	
	# extend the data store
	sentiment_data.extend(temp_data)
	
	# print the progress
	str_print = "got data from {:s} to {:s}".format(*(sentiment_data[0]["datetime"],
	                                                sentiment_data[-1]["datetime"],))
	print(str_print)
	
	# sleep
	time.sleep(.02)




ReadTimeout: HTTPConnectionPool(host='api-dev.augmento.ai', port=80): Read timed out. (read timeout=10)

In [ ]:
# save the data
print("saving data to {:s}".format(filename_save_data))
with open(filename_save_data, "wb") as f:
	f.write(zlib.compress(msgpack.packb(sentiment_data)))


print("done!")